# Nuestra primer aplicacion

## Atencion
Para ejecutar esta libreta primero tienen que configurar su sistema como se describe en [la libreta de preparacion](./L00_Preparacion.ipynb). Comenzaremos por conectarnos al dispositivo y asegurar que podemos comunicarnos con el.

In [1]:
!adb devices
!adb forward tcp:9999 tcp:2222

List of devices attached
R88DEAEY55Y59THU	device



In [2]:
import android

In [3]:
andy = android.Android()
andy.ttsSpeak("listo para recibir instrucciones|")

Result(id=0, result=None, error=None)

## Objetivo
En la sesion de hoy tendremos como objetivo escribir una aplicacion que se conecte a un servicio externo y obtenga la informacion del estado del tiempo en la posicion actual del dispositivo, y finalmente que el dispositivo "hable" el estado del tiempo descargado. Para esto necesitaremos:

1. conocer la ubicacion del dispositivo
2. solcitar la informacion del estado del tiempo en la ubicacion del dispositivo

Para 1. usaremos la fachada de localizacion , en particular usaremos [getLastKnownLocation](https://github.com/kuri65536/sl4a/blob/master/docs/ApiReference.md#getlastknownlocation) y [geocode](https://github.com/kuri65536/sl4a/blob/master/docs/ApiReference.md#geocode).
Para 2. usaremos la informacion obtenida de 1. y algunas librerias externas de Python para solicitar el estado del tiempo usando la API de [OpenWeatherMap](https://openweathermap.org/city/3526617).

Comenzamos!

## 0. Obtener geolocalización

In [4]:
andy.startLocating()
andy.readLocation().result
andy.stopLocating()

Result(id=3, result=None, error=None)

## 1. Determinar ultima posicion conocida del dispositivo

In [5]:
# obtener la ultima localizacion conocida
andy.getLastKnownLocation().result

{u'gps': None,
 u'network': {u'accuracy': 22.808000564575195,
  u'altitude': 0,
  u'bearing': 0,
  u'latitude': 19.5232633,
  u'longitude': -96.9306004,
  u'provider': u'network',
  u'speed': 0,
  u'time': 1522092232037},
 u'passive': {u'accuracy': 22.808000564575195,
  u'altitude': 0,
  u'bearing': 0,
  u'latitude': 19.5232633,
  u'longitude': -96.9306004,
  u'provider': u'network',
  u'speed': 0,
  u'time': 1522092232037}}

In [6]:
# asignamos la ultima localizacion conocida a una variable.
# notemos que la variable es del tipo diccionario (dict)
ultimaLocalizacionConocida = andy.getLastKnownLocation().result

type(ultimaLocalizacionConocida)

dict

In [7]:
# notemos que el valor de la llave 'gps' es nulo
print(ultimaLocalizacionConocida['gps'])

None


In [8]:
# accedemos al valor de la llave network
print(ultimaLocalizacionConocida['network'])

{u'bearing': 0, u'altitude': 0, u'provider': u'network', u'longitude': -96.9306004, u'time': 1522092232037, u'latitude': 19.5232633, u'speed': 0, u'accuracy': 22.808000564575195}


El valor asignado a la variable `ultimaLoc` dependera de cual de los proveedores sea no nulo. Al final de la celda desplegamos el contenido asignado a `ultimaLoc`. Noten la estructura de diccionario.

In [9]:
# asignamos una variable al contenido de la llave que no es nula usando una condicional

if ultimaLocalizacionConocida['gps'] is not None:
    ultimaLoc = ultimaLocalizacionConocida['gps']
else:
    ultimaLoc = ultimaLocalizacionConocida['network']

print type(ultimaLoc)
print ultimaLoc
ultimaLoc.keys()

<type 'dict'>
{u'bearing': 0, u'altitude': 0, u'provider': u'network', u'longitude': -96.9306004, u'time': 1522092232037, u'latitude': 19.5232633, u'speed': 0, u'accuracy': 22.808000564575195}


[u'bearing',
 u'altitude',
 u'provider',
 u'longitude',
 u'time',
 u'latitude',
 u'speed',
 u'accuracy']

Ahora vamos a extraer la latitud, longitud y estampa temporal del diccionario anterior. 

In [10]:
# del diccionario extraemos latitud y longitud y fecha
lat = ultimaLoc["latitude"]
lon = ultimaLoc["longitude"]
time = ultimaLoc["time"]
print time

1522092232037


Noten que la variable `time` es lo que se conoce como un `timestamp` un numero entero que cuenta los segundos transcurridos desde un inicio determinado. De este modo necesitamos convertirla en un formato amigable. Para esto usamos un modulo de Python que se llama `datetime`

In [11]:
import datetime

fecha = datetime.datetime.fromtimestamp(time / 1e3)
print fecha

2018-03-26 13:23:52.037000


In [12]:
# otra forma de presentar la fecha solo hasta segundos
fecha = datetime.datetime.fromtimestamp(time/1e3).strftime('%Y-%m-%d %H:%M:%S')
print fecha

2018-03-26 13:23:52


In [13]:
hora = datetime.datetime.fromtimestamp(time/1e3).strftime('%H:%M:%S')
print hora

13:23:52


Ahora usaremos [geocode](https://github.com/kuri65536/sl4a/blob/master/docs/ApiReference.md#geocode) que es una funcion del dispositivo que regresa una lista de direcciones para la latitud y longitud dadas:

In [14]:
andy.geocode(lat,lon,5).result

[{u'admin_area': u'Veracruz',
  u'country_code': u'MX',
  u'country_name': u'M\xe9xico',
  u'feature_name': u'54',
  u'locality': u'Xalapa Enr\xedquez',
  u'postal_code': u'91070',
  u'sub_admin_area': u'Xalapa',
  u'thoroughfare': u'Avenida Venustiano Carranza'},
 {u'admin_area': u'Veracruz',
  u'country_code': u'MX',
  u'country_name': u'M\xe9xico',
  u'feature_name': u'23',
  u'locality': u'Xalapa Enr\xedquez',
  u'postal_code': u'91070',
  u'sub_admin_area': u'Xalapa',
  u'thoroughfare': u'Calle Centro America'},
 {u'admin_area': u'Veracruz',
  u'country_code': u'MX',
  u'country_name': u'M\xe9xico',
  u'feature_name': u'Francisco I. Madero',
  u'locality': u'Xalapa Enr\xedquez',
  u'postal_code': u'91070',
  u'sub_admin_area': u'Xalapa'},
 {u'admin_area': u'Veracruz',
  u'country_code': u'MX',
  u'country_name': u'M\xe9xico',
  u'feature_name': u'Xalapa Enr\xedquez',
  u'locality': u'Xalapa Enr\xedquez',
  u'sub_admin_area': u'Xalapa'},
 {u'admin_area': u'Veracruz',
  u'country_co

Noten que el resultado no es un diccionario, sino una lista de diccionarios. Si no se especifica el ultimo numero (cuantas direcciones solicitar) se regresa solo un valor:

In [15]:
direccion = andy.geocode(lat,lon).result
print direccion
type(direccion)

[{u'thoroughfare': u'Avenida Venustiano Carranza', u'locality': u'Xalapa Enr\xedquez', u'sub_admin_area': u'Xalapa', u'admin_area': u'Veracruz', u'feature_name': u'54', u'country_code': u'MX', u'country_name': u'M\xe9xico', u'postal_code': u'91070'}]


list

Dado que la variable `direccion` es una lista, debemos acceder a sus elementos especificando un indice:

In [16]:
codigoPostal = direccion[0]["postal_code"]
print codigoPostal
codigoPais = direccion[0]["country_code"]
print codigoPais

91070
MX


## 2. Solicitar estado del tiempo usando API de OpenWeatherMap

Con el codigo postal y el codigo del pais podemos solicitar informacion a la API en la siguiente url:

In [17]:
url="http://api.openweathermap.org/data/2.5/weather?zip={0},{1}&APPID=dfeea5e604e1948b28652216ad910d11".format(codigoPostal,codigoPais)

In [18]:
print(url)

http://api.openweathermap.org/data/2.5/weather?zip=91070,MX&APPID=dfeea5e604e1948b28652216ad910d11


Podemos solicitar la informacion a un `url` directamente usando una libreria que se llama `requests` del siguiente modo:

In [19]:
import requests
estadoDelTiempo = requests.get(url).json()

# vamos a usar urrlib2 pues requests no esta disponible en el movil

import urllib2
response = urllib2.urlopen(url)
respuesta = response.read()
response.close()

In [20]:
# convertimos el resultado a un dictionario de JSON

import json

estadoDelTiempo = json.loads(respuesta)

El resultado es un diccionario en formato json

In [21]:
estadoDelTiempo

{u'base': u'stations',
 u'clouds': {u'all': 8},
 u'cod': 200,
 u'coord': {u'lat': 19.51, u'lon': -97.1},
 u'dt': 1522092222,
 u'id': 590134798,
 u'main': {u'grnd_level': 866.37,
  u'humidity': 37,
  u'pressure': 866.37,
  u'sea_level': 1023.52,
  u'temp': 299.491,
  u'temp_max': 299.491,
  u'temp_min': 299.491},
 u'name': u'Xalapa',
 u'sys': {u'country': u'MX',
  u'message': 0.0031,
  u'sunrise': 1522067193,
  u'sunset': 1522111271},
 u'weather': [{u'description': u'clear sky',
   u'icon': u'02d',
   u'id': 800,
   u'main': u'Clear'}],
 u'wind': {u'deg': 40.503, u'speed': 1.11}}

In [22]:
estadoDelTiempo['name']

u'Xalapa'

In [23]:
estadoDelTiempo.keys()

[u'clouds',
 u'name',
 u'coord',
 u'sys',
 u'weather',
 u'cod',
 u'base',
 u'dt',
 u'main',
 u'id',
 u'wind']

In [24]:
# queremos la descripcion de estado del tiempo
estadoDelTiempo["weather"]

[{u'description': u'clear sky',
  u'icon': u'02d',
  u'id': 800,
  u'main': u'Clear'}]

In [25]:
# es una lista
descripcion = estadoDelTiempo["weather"][0]['description']

In [26]:
import sys
reload(sys)  # Reload does the trick!
sys.setdefaultencoding('UTF8')

# Ejercicio

Completar los valores que hacen falta entre las comas al final de la siguiente instruccion.
Necesitan ingresar cuatro valores entre las comas, los valores estan en alguna de las celdas de evaluacion de esta libreta.
El mensaje debe verse como sigue:

    'En la ciudad de Xalapa Enr\xc3\xadquez, siendo las 2018-03-19 13:09:32 horas, en la calle de Avenida Enrique C. R\xc3\xa9bsamen, el estado del tiempo es scattered clouds'

In [44]:
# ejercicio, completa los valores que hacen falta entre las comas
mensaje='En la ciudad de {0}, siendo las {1} horas, en la calle {2}, el estado del tiempo es {3}'.format( \
                                estadoDelTiempo['name'],hora,direccion[0]["thoroughfare"],estadoDelTiempo["weather"][0]["description"])

In [41]:
mensaje

'En la ciudad de Xalapa, siendo las 13:23:52 horas, en la calle de Avenida Venustiano Carranza, el estado del tiempo es clear sky'

Finalmente le damos al dispositivo la instruccion de darnos el mensaje creado...

In [45]:
andy.ttsSpeak(mensaje)

Result(id=13, result=None, error=None)

Felicidades! han hecho su primer app en Python para Android. En la siguiente sesion vamos a integrar la app con la funcionalidad de localizacion para que pueda decirles el estado del tiempo en cualquier lugar en donde se encuentren usando la posicion actual.